# 목표 PEIS점수 내기 & 안쓰는 기업년도 다 빼기

# 필요 모듈

In [4]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 엑셀읽기

In [6]:
df = pd.read_excel("Data/kospi/KospiPEISData&PriceData.xlsx")

# 1. 안쓰는거 (NULL값 20개 이상) 제거

In [7]:
df = Cut20NullfillNA(df)

# 2. PEIS를 구하기 위한 기초데이터들 연산해보기 
## (한꺼번에 계산해서 앞년 뒷년버리기)

In [8]:
PEISBaseData = CompuMoreData(df)
PEISBaseData = setSGA(PEISBaseData)
PEISBaseData.to_excel('temp/kospi/PEISBaseCompu.xlsx',index=False)

# 3. Drop데이터들 Drop 시키기
1. 회계년 08 09 20 21은 사용 불가능함 
2. deltaprice 0 99 inf는 연산중에 나온 오류
3. PEIS 점수 에러나는 것들은 좀 특이케이스도 있고 상폐전후로 생기는 에러가 있음 특이케이스는 얼마안대서 괜찮은듯

In [9]:
dropdata(PEISBaseData, '회계년', [2008, 2009, 2020, 2021])
dropdata(PEISBaseData, 'deltaprice', [0, 99, np.inf])
dropdata(PEISBaseData, 'deltaACC', [np.inf, -np.inf])
dropdata(PEISBaseData, 'RNOA', [0]) # 그냥 연도 넘어가면서 이상해지는 것들 제거 
dropdata(PEISBaseData, 'SGAp', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
dropdata(PEISBaseData, 'deltaGM1', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
print(PEISBaseData.shape)

(7320, 52)


# 4. 가공된 데이터 토대로 +1,-1,0 구하기 (년도별로)

In [10]:
Yearlydfs = yearbaseDFs(PEISBaseData, years =[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
PEISBaseData = CalPEISpoints(Yearlydfs)
PEISBaseData.shape

(7320, 59)

# 5. PEIS 점수 구하기 (단순 행 sum)

In [11]:
PEISBaseData = CalPEIS(PEISBaseData)

# 6. 저장하기
* PEIS,Stock 계산확인용 엑셀 하나 필요
* 년도 적절한지 + 값이 정확히 계산되어있는지 특수케이스(중간 상폐,상장 정도만 확인하면 댈듯? 앞뒤년도 짜르고)

In [12]:
PEISBaseData.to_excel('Data/kospi/Alldata.xlsx', index = False)

#

#

#

#

#

#

# assert 공간 아래는 포트폴리오 부분에서 구현하는 게 좋을 것 같음 class랑 Stat은 따로 만드는게 좋을 듯 

# 그리고 모든게 괜찮다고 판단되면 코스닥도 해줘야함

In [13]:
test = PEISBaseData[['SGAdeltasale>0PEISpoint','SGAdeltasale<0PEISpoint','GNOAPEISpoint','deltaATOPEISpoint','deltaGMPEISpoint','deltaACCPEISpoint','RNOAPEISpoint']]!=0

In [14]:
test.sum()

SGAdeltasale>0PEISpoint    1840
SGAdeltasale<0PEISpoint    1108
GNOAPEISpoint              2936
deltaATOPEISpoint          2936
deltaGMPEISpoint           2936
deltaACCPEISpoint          2936
RNOAPEISpoint              2936
dtype: int64

# 코스닥

# 0. 엑셀읽기

In [15]:
df = pd.read_excel("Data/kosdaq/KosdaqPEISData&PriceData.xlsx")

# 1. 안쓰는거 (NULL값 20개 이상) 제거

In [16]:
df = Cut20NullfillNA(df)

# 2. PEIS를 구하기 위한 기초데이터들 연산해보기 
## (한꺼번에 계산해서 앞년 뒷년버리기)

In [17]:
PEISBaseData = CompuMoreData(df)
PEISBaseData = setSGA(PEISBaseData)
PEISBaseData.to_excel('temp/kosdaq/PEISBaseCompu.xlsx',index=False)

# 3. Drop데이터들 Drop 시키기
1. 회계년 08 09 20 21은 사용 불가능함 
2. deltaprice 0 99 inf는 연산중에 나온 오류
3. PEIS 점수 에러나는 것들은 좀 특이케이스도 있고 상폐전후로 생기는 에러가 있음 특이케이스는 얼마안대서 괜찮은듯

In [18]:
dropdata(PEISBaseData, '회계년', [2008, 2009, 2020, 2021])
dropdata(PEISBaseData, 'deltaprice', [0, 99, np.inf])
dropdata(PEISBaseData, 'deltaACC', [np.inf, -np.inf])
dropdata(PEISBaseData, 'RNOA', [0]) # 그냥 연도 넘어가면서 이상해지는 것들 제거 
dropdata(PEISBaseData, 'SGAp', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
dropdata(PEISBaseData, 'deltaGM1', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
print(PEISBaseData.shape)

(10970, 52)


# 4. 가공된 데이터 토대로 +1,-1,0 구하기 (년도별로)

In [19]:
Yearlydfs = yearbaseDFs(PEISBaseData, years =[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
PEISBaseData = CalPEISpoints(Yearlydfs)
PEISBaseData.shape

(10970, 59)

# 5. PEIS 점수 구하기 (단순 행 sum)

In [20]:
PEISBaseData = CalPEIS(PEISBaseData)

# 6. 저장하기
* PEIS,Stock 계산확인용 엑셀 하나 필요
* 년도 적절한지 + 값이 정확히 계산되어있는지 특수케이스(중간 상폐,상장 정도만 확인하면 댈듯? 앞뒤년도 짜르고)

In [21]:
PEISBaseData.to_excel('Data/kosdaq/Alldata.xlsx', index = False)